
<a href="http://landlab.github.io"><img style="float: left; width: 300px;" src="https://landlab.csdms.io/_static/landlab_logo.png"></a>

# 2D Surface Water Flow Component: Green River Case Study

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>

## Overview

This notebook demonstrates the usage of the `RiverFlowDynamics` Landlab component to simulate surface water flow over real terrain. Unlike theoretical examples with idealized topography, this example uses an actual digital elevation model (DEM) of a specific reach of the Green River in Utah, USA.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from tqdm import trange
from landlab import RasterModelGrid
from landlab.components import RiverFlowDynamics
from landlab.io import esri_ascii
from landlab.plot.imshow import imshow_grid

## Loading the Digital Elevation Model

We'll load the DEM of the Green River reach from an ASCII file. This file contains elevation data in a grid format that Landlab can process.

In [ ]:
# Getting DEM parameters - Select one of the following DEM
#asc_file = "DEM_GreenRiver_100x98_20x20.asc" # 20 by 20 meters resolution DEM
asc_file = "DEM_GreenRiver_80x78_25x25.asc" # 25 by 25 meters resolution DEM

# Read the DEM file
with open(asc_file) as f:
    grid = esri_ascii.load(f, name="topographic__elevation")

# Get the topographic elevation field
teDEM = grid.at_node["topographic__elevation"]

## Define Simulation Parameters

We'll specify basic parameters such as Manning's roughness coefficient, time step duration, and the number of time steps for our simulation.

In [ ]:
# Basic parameters
n = 0.034  # Manning's roughness coefficient, [s/m^(1/3)]

# Simulation parameters
n_timesteps = 400  # Number of timesteps
dt = 1.0  # Timestep duration, [s]

# Display parameters
display_animation_freq = 5  # How often to display animation frames

## Set Up Topography

The DEM may contain no-data values (typically -9999), which we need to replace with reasonable values. We'll replace these with an elevation of 700m.

In [ ]:
# The DEM is already loaded, so just replace -9999 (no-data values) with 700
teDEM = grid.at_node["topographic__elevation"]  # Access the existing field
grid.at_node["topographic__elevation"] = np.where(teDEM == -9999, 700, teDEM)  # Update it

### Visualize Initial Topography

Let's visualize the topography to get a better understanding of the terrain we're working with.

In [ ]:
# Showing the topography
plt.figure(figsize=(12, 10))
im = imshow_grid(grid, "topographic__elevation", cmap="terrain")
plt.colorbar(im, label="Elevation (m)")
plt.title("Green River Topography")
plt.show()

## Set Initial Conditions

To begin our simulation with an empty channel, we need to create three essential fields:
1. Water depth: Initially set to zero at all nodes
2. Water velocity: Initially set to zero at all links
3. Water surface elevation: Calculated as the sum of topographic elevation and water depth (initially equal to just the topography)

In [ ]:
# We establish the initial conditions, which represent an empty channel
h = grid.add_zeros("surface_water__depth", at="node")

# Water velocity is zero in everywhere since there is no water yet
vel = grid.add_zeros("surface_water__velocity", at="link")

# Calculating the initial water surface elevation from water depth and topographic elevation
wse = grid.add_zeros("surface_water__elevation", at="node")
wse += h + grid.at_node["topographic__elevation"]  # Use the existing field directly

## Set Boundary Conditions

We need to specify where water enters the domain. For this simulation, water will flow from right to left, so we'll set the nodes at the right edge as our entry points.

In [ ]:
# We set fixed boundary conditions, specifying the nodes and links in which the water is flowing into the grid
fixed_entry_nodes = grid.nodes_at_right_edge
fixed_entry_links = grid.links_at_node[fixed_entry_nodes][:, 2]

### Define Entry Flow Conditions

Now we need to set the water depth and velocity at the entry points. We'll simulate an average discharge of 247 m³/s with a water depth of approximately 5.46 m in the deepest part of the channel (thalweg).

In [ ]:
# We set the fixed values in the entry nodes/links
# (Average discharge 247 m3/s and water depth of 5.4559 m in the thalweg)
thalweg = grid.at_node["topographic__elevation"][fixed_entry_nodes].min()
WSE = thalweg + 5.4559  # Water surface elevation

# Calculate water depth at each entry node
entry_nodes_h_values = WSE - grid.at_node["topographic__elevation"][fixed_entry_nodes]
entry_nodes_h_values = np.where(entry_nodes_h_values < 0, 0, entry_nodes_h_values)

# Calculate water velocity at entry links
tempB1 = (entry_nodes_h_values > 0)
tempCalc1 = np.where(tempB1 == True, entry_nodes_h_values, 1)
entry_links_vel_values = np.where(tempB1 == True, (247/(grid.dx*np.sum(tempB1)))*(1/tempCalc1), 0)

### Visualize Entry Cross-section

Let's plot the entry cross-section to visualize the water depth and terrain at the point where water enters the domain.

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(
    grid.y_of_node[fixed_entry_nodes], 
    entry_nodes_h_values + grid.at_node["topographic__elevation"][fixed_entry_nodes],
    'b-', linewidth=2, label="Water Surface"
)
plt.plot(
    grid.y_of_node[grid.nodes_at_right_edge], 
    grid.at_node["topographic__elevation"][grid.nodes_at_right_edge],
    'k-', linewidth=2, label="Terrain"
)
plt.fill_between(
    grid.y_of_node[fixed_entry_nodes],
    grid.at_node["topographic__elevation"][fixed_entry_nodes],
    entry_nodes_h_values + grid.at_node["topographic__elevation"][fixed_entry_nodes],
    color='blue', alpha=0.3
)
plt.title("Entry Cross-section of Green River", fontsize=14)
plt.xlabel("Distance Along Cross-section [m]", fontsize=12)
plt.ylabel("Elevation [m]", fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

## Initialize the RiverFlowDynamics Component

Now we'll create the `RiverFlowDynamics` component with our parameters and boundary conditions.

In [ ]:
# Initialize the model with our parameters and boundary conditions
rfd = RiverFlowDynamics(
    grid,
    dt=dt,
    mannings_n=n,
    fixed_entry_nodes=fixed_entry_nodes,
    fixed_entry_links=fixed_entry_links,
    entry_nodes_h_values=entry_nodes_h_values,
    entry_links_vel_values=entry_links_vel_values,
)

## Run the Simulation

Now we'll run the model for the specified number of time steps, visualizing the water depth at regular intervals to see how the river fills the channel.

In [ ]:
# Setting up the figure for animation
plt.figure(figsize=(12, 10))

for timestep in range(n_timesteps):
    rfd.run_one_step()
    
    if timestep % display_animation_freq == 0:
        clear_output(wait=True)  # Clear the previous image
        plt.figure(figsize=(12, 10))
        im = imshow_grid(grid, "surface_water__depth", cmap="Blues")
        plt.colorbar(im, label="Water Depth (m)")
        plt.title(f"Water Depth at Time Step {timestep+1} (Time: {(timestep+1)*dt}s)")
        plt.show()

## Visualize Final Results

Let's examine the water depth and water surface elevation at the end of the simulation.

In [ ]:
# Final water depth
plt.figure(figsize=(12, 10))
im = imshow_grid(grid, "surface_water__depth", cmap="Blues")
plt.colorbar(im, label="Water Depth (m)")
plt.title(f"Final Water Depth after {n_timesteps} Time Steps ({n_timesteps*dt}s)")
plt.show()

In [ ]:
# Final water surface elevation
plt.figure(figsize=(12, 10))
im = imshow_grid(grid, "surface_water__elevation", cmap="terrain")
plt.colorbar(im, label="Water Surface Elevation (m)")
plt.title(f"Final Water Surface Elevation after {n_timesteps} Time Steps ({n_timesteps*dt}s)")
plt.show()

## Visualize Flow Velocity Magnitude

Let's calculate and visualize the flow velocity magnitude across the domain.

In [ ]:
# Extract velocity components
links_at_node = grid.links_at_node
velocity_at_links = grid.at_link["surface_water__velocity"]

# Calculate velocity magnitude at nodes (approximate)
velocity_magnitude = np.zeros(grid.number_of_nodes)
for i in range(grid.number_of_nodes):
    # Get valid links (those that are not -1)
    valid_links = links_at_node[i][links_at_node[i] >= 0]
    if len(valid_links) > 0:
        # Take the maximum velocity of connected links as an approximation
        velocity_magnitude[i] = np.max(np.abs(velocity_at_links[valid_links]))

# Add velocity magnitude field to grid
grid.at_node["velocity_magnitude"] = velocity_magnitude

# Plot velocity magnitude
plt.figure(figsize=(12, 10))
im = imshow_grid(grid, "velocity_magnitude", cmap="plasma")
plt.colorbar(im, label="Velocity Magnitude (m/s)")
plt.title(f"Flow Velocity Magnitude after {n_timesteps} Time Steps")
plt.show()

## Analyze Cross-sections

Let's examine the water depth and flow along cross-sections of the domain to better understand the simulation results.

In [ ]:
# Find the middle of the domain for cross-sections
mid_row = grid.shape[0] // 2
mid_col = grid.shape[1] // 2

# Extract cross-section through the middle row
row_nodes = np.arange(mid_row * grid.shape[1], (mid_row + 1) * grid.shape[1], dtype=int)

# Extract cross-section through the middle column
col_nodes = np.array([mid_col + i * grid.shape[1] for i in range(grid.shape[0])], dtype=int)

# Plot horizontal cross-section
plt.figure(figsize=(14, 6))

# Plot terrain
plt.plot(grid.x_of_node[row_nodes], grid.at_node['topographic__elevation'][row_nodes], 
         'k-', linewidth=2, label='Terrain')

# Plot water surface
plt.plot(grid.x_of_node[row_nodes], grid.at_node['surface_water__elevation'][row_nodes], 
         'b-', linewidth=2, label='Water Surface')

# Fill between terrain and water surface where there's water
water_indices = grid.at_node['surface_water__depth'][row_nodes] > 0
plt.fill_between(grid.x_of_node[row_nodes][water_indices], 
                 grid.at_node['topographic__elevation'][row_nodes][water_indices],
                 grid.at_node['surface_water__elevation'][row_nodes][water_indices],
                 color='blue', alpha=0.3)

plt.grid(True)
plt.xlabel('Distance Along Cross-section (m)', fontsize=12)
plt.ylabel('Elevation (m)', fontsize=12)
plt.title('Longitudinal Cross-Section (East-West) through Domain Center', fontsize=14)
plt.legend()
plt.show()

# Plot vertical cross-section 
plt.figure(figsize=(14, 6))

# Plot terrain
plt.plot(grid.y_of_node[col_nodes], grid.at_node['topographic__elevation'][col_nodes], 
         'k-', linewidth=2, label='Terrain')

# Plot water surface
plt.plot(grid.y_of_node[col_nodes], grid.at_node['surface_water__elevation'][col_nodes], 
         'b-', linewidth=2, label='Water Surface')

# Fill between terrain and water surface where there's water
water_indices = grid.at_node['surface_water__depth'][col_nodes] > 0
plt.fill_between(grid.y_of_node[col_nodes][water_indices], 
                 grid.at_node['topographic__elevation'][col_nodes][water_indices],
                 grid.at_node['surface_water__elevation'][col_nodes][water_indices],
                 color='blue', alpha=0.3)

plt.grid(True)
plt.xlabel('Distance Along Cross-section (m)', fontsize=12)
plt.ylabel('Elevation (m)', fontsize=12)
plt.title('Transverse Cross-Section (North-South) through Domain Center', fontsize=14)
plt.legend()
plt.show()

## Calculate Discharge at Different Cross-sections

Let's calculate the discharge at several cross-sections along the river to see how it changes throughout the reach.

In [ ]:
# Define cross-section positions (as percentages of domain length from right edge)
cross_section_positions = [0.1, 0.3, 0.5, 0.7, 0.9]  # 10%, 30%, 50%, 70%, 90%
discharge_values = []

for pos in cross_section_positions:
    # Calculate column index
    col_idx = int((1 - pos) * (grid.shape[1] - 1))
    
    # Get nodes along this column
    col_nodes = np.array([col_idx + i * grid.shape[1] for i in range(grid.shape[0])], dtype=int)
    
    # Calculate discharge (sum of depth * velocity * cell width for all nodes in the cross-section)
    discharge = 0
    for node in col_nodes:
        # Get the horizontal (west-east) links connected to this node
        links = grid.links_at_node[node]
        east_link = links[1]  # Index 1 corresponds to the eastern link
        
        if east_link >= 0:  # Check if link exists
            depth = grid.at_node['surface_water__depth'][node]
            velocity = grid.at_link['surface_water__velocity'][east_link]
            discharge += depth * velocity * grid.dy
    
    discharge_values.append(abs(discharge))  # Take absolute value for consistent sign

# Plot discharge along the river
plt.figure(figsize=(10, 6))
plt.plot(np.array(cross_section_positions) * 100, discharge_values, 'bo-', linewidth=2, markersize=8)
plt.grid(True)
plt.xlabel('Position Along River (% from upstream)', fontsize=12)
plt.ylabel('Discharge (m³/s)', fontsize=12)
plt.title('Discharge at Different Cross-sections', fontsize=14)
plt.show()

## Physics of River Flow in Natural Channels

This simulation demonstrates several key physical processes that govern the flow of water in natural river channels:

1. **Channel Hydraulics**: The flow is controlled by the balance between gravity (driving force) and friction (resistive force). Water accelerates in steeper sections and slows down in flatter areas.

2. **Channel Morphology Effects**: The irregular shape of natural channels creates complex flow patterns with varying depths and velocities. Deeper sections typically have higher velocities, while shallow areas may experience slower flow.

3. **Conservation Principles**: Throughout the simulation, mass (water volume) and momentum are conserved, ensuring physical realism.

4. **Boundary Interactions**: The simulation captures how water interacts with channel boundaries, including bank effects and flow acceleration in constricted reaches.

In real rivers, additional factors like sediment transport, vegetation effects, and temporal variations in discharge would further influence the flow dynamics. The `RiverFlowDynamics` component provides a solid foundation for understanding these complex interactions.

## Conclusion

In this tutorial, we've demonstrated how to use the `RiverFlowDynamics` component in Landlab to simulate surface water flow over real terrain. We've seen how water fills a natural river channel, creating complex flow patterns influenced by topography.

Some key insights from this simulation:

1. **Realistic Flow Patterns**: The simulation captures the natural tendency of water to follow the path of least resistance, with higher depths in depressions and faster velocities in constricted areas.

2. **Inflow-Outflow Balance**: Over time, the flow through the domain stabilizes as inflow matches outflow, reaching a steady-state condition that reflects the real-world behavior of the river.

3. **Topographic Control**: The pre-existing terrain exerts strong control on the flow patterns, demonstrating how river morphology is fundamentally shaped by underlying topography.

4. **Manning's Roughness Effects**: The choice of Manning's coefficient (n = 0.012) represents a relatively smooth channel, appropriate for a major river like the Green River. Different coefficients would result in different flow patterns and water depths.

Potential extensions to this example include:
- Adding variable roughness based on land cover or channel substrate
- Simulating flood events by increasing the inflow discharge
- Implementing sediment transport and morphological evolution
- Adding structures like bridges or dams to study their hydraulic impacts

-- --
### And that's it! 

Nice work completing this tutorial. You now know how to use the `RiverFlowDynamics` Landlab component to run simulations with real topographic data :)

-- --